#This is the last and final implementation on the BraTS data set. This is an implementation of a Random Forest. The author of this model can be found here. https://github.com/dgriffiths3/ml_segmentation/blob/master/train.py

The decision to use a Random Forest was necessary since traditional machine learning is still a valid path in predictive analysis, especially when neural nets can still be considered faulty. The impetus for returning to traditional machine learning methods can be found here. 
https://www.youtube.com/channel/UC34rW-HtPJulxr5wp2Xa04w/videos 

This notebook explores the steps needed to convert an MRI training and testing set into a data set for a Random Forest and then finally training on the model. 

NOTE: If you wish to train on this MRI set, which is >50GB, it will be very costly in time if you decide to use Colab, even with the use of a GPU. Creating a notebook instance in AWS with several GPUs is recommended. It will come at some fiscal cost but it will save you time.


This is the code for mounting your notebook to your google drive directory.

In [0]:
# mount google drive for access and file manipulation 
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


The following imports are necessary to proceed with the feature engineering and modeling process. 

In [0]:
pip install mahotas

     |████████████████████████████████| 1.5MB 3.4MB/s 
  Created wheel for mahotas: filename=mahotas-1.4.9-cp36-cp36m-linux_x86_64.whl size=4264081 sha256=0b17ff6be95675e570c4b5786a7c795c3ffbacffc64dcf3c99d75d41d8422136
  Stored in directory: /root/.cache/pip/wheels/02/e7/50/b67eee3b19c479cc5b41f10230bf9c1556be6eaaf095c6e997
Successfully built mahotas


In [0]:
import cv2
import numpy as np
import pylab as plt
from glob import glob
import argparse
import os
import progressbar
import pickle as pkl
from numpy.lib import stride_tricks
from skimage import feature
from sklearn import metrics
from sklearn.model_selection import train_test_split
import time
import mahotas as mt

This method is for sampling. We do this by randomly generating indices that correspond to pixels in the image. 

In [0]:
def subsample_idx(low, high, sample_size):

    return np.random.randint(low,high,sample_size)

Haralick features are obtained using the mahotas import. This technique utilizes the adjacency matrix and stores the count of uniquely paired pixels. For an image matrix containing only 3 values. The adjacency matrix will be 3x3 and if values 0-1 neighboring each 20 times in a 10x10 matrix, value 20 will be located in position (i,j) = (0,1) in the adjacency matrix. This approach is useful for anomaly detection. 

In [0]:
def calc_haralick(roi):
    feature_vec = []

    texture_features = mt.features.haralick(roi)
    mean_ht = texture_features.mean(axis=0)

    [feature_vec.append(i) for i in mean_ht[0:9]]

    return np.array(feature_vec)

In [0]:
################################################################
def haralick_features(img, h_neigh, ss_idx):
    print ('Computing haralick features.')

    size = h_neigh
    shape = (img.shape[0] - size + 1, img.shape[1] - size + 1, size, size)
    strides = 2 * img.strides
    patches = stride_tricks.as_strided(img, shape=shape, strides=strides)
    patches = patches.reshape(-1, size, size)

    if len(ss_idx) == 0 :
        bar = progressbar.ProgressBar(maxval=len(patches), \
        widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
        
    else:
        bar = progressbar.ProgressBar(maxval=len(ss_idx), \
        widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

    bar.start()

    h_features = []

    if len(ss_idx) == 0:
        for i, p in enumerate(patches):
            bar.update(i+1)
            h_features.append(calc_haralick(p))

    else:
        for i, p in enumerate(patches[ss_idx]):
            bar.update(i+1)
            h_features.append(calc_haralick(p))

    #h_features = [calc_haralick(p) for p in patches[ss_idx]]

    return np.array(h_features)

This method is for selects a single model based on the classifier parameter. 

In [0]:
def train_model(X, y, classifier):
    if classifier == "SVM":
        from sklearn.svm import SVC
        print ('Training Support Vector Machine model.')
        model = SVC()
        model.fit(X, y)

    elif classifier == "RF":
        from sklearn.ensemble import RandomForestClassifier
        print ('Training Random Forest model.')
        model = RandomForestClassifier(n_estimators=250, max_depth=12, random_state=42)
        model.fit(X, y)
        
    elif classifier == "GBC":
        from sklearn.ensemble import GradientBoostingClassifier
        model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
        model.fit(X, y)

    print ('Model training complete.')
    print ('Training Accuracy: %.2f' %model.score(X, y))

    return model

This method is meant for prediction and metrics. It will print all of the pertinent metrics: accuracy, precision, recall and f1. 

In [0]:
def test_model(X, y, model):
    pred = model.predict(X)
    precision = metrics.precision_score(y, pred, average='weighted', labels=np.unique(pred))
    recall = metrics.recall_score(y, pred, average='weighted', labels=np.unique(pred))
    f1 = metrics.f1_score(y, pred, average='weighted', labels=np.unique(pred))
    accuracy = metrics.accuracy_score(y, pred)

    print ('--------------------------------')
    print ('Accuracy: %.2f' %accuracy)
    print ('Precision: %.2f' %precision)
    print ('Recall: %.2f' %recall)
    print ('F1: %.2f' %f1)
    print ('--------------------------------')

In [0]:
cd /content/drive/My Drive/brats_png

/content/drive/My Drive/brats_png


In [0]:
ls

images/  labels/


Get the filepaths to the images and labels. Note: these pngs are preprocessed images from the Unet model in the same repository. The name of notebook is brats_processing

In [0]:
i = '/content/drive/My Drive/brats_png/images/' 
l = '/content/drive/My Drive/brats_png/labels/'

In [0]:
filelist1 = glob(os.path.join(i, '*.png'))

Checking the png list

In [0]:
filelist1[:5]

['/content/drive/My Drive/brats_png/images/pat419_0001_10.png',
 '/content/drive/My Drive/brats_png/images/pat419_0001_11.png',
 '/content/drive/My Drive/brats_png/images/pat419_0001_12.png',
 '/content/drive/My Drive/brats_png/images/pat419_0001_13.png',
 '/content/drive/My Drive/brats_png/images/pat419_0001_14.png']

tdqm is a great tool for displaying progress. You can use it in the for loop to enumerate. 

In [0]:
from tqdm import tqdm

Here we read in the pngs and convert to arrays via cv2 and append to the image_list

In [0]:
 image_list = []
 for file1 in tqdm(filelist1[100:2100]):
        #image_list.append(cv2.imread(file, 1))
        i = cv2.imread(file1)
        #print(i[i > 0])
        image_list.append(i)

#start time
#finish time

In [0]:
#filelist2 = glob(os.path.join(label_dir, '*.png'))
filelist2 = glob(os.path.join(l, '*.png'))

In [0]:
label_list = []
for file2 in tqdm(filelist2[100:2100]):
  #label_list.append(cv2.imread(file, 1))
  j = cv2.imread(file2)
  #print(j[j>0])
  label_list.append(j)

In [0]:
filelist2[:5]

['/content/drive/My Drive/brats_png/labels/mask_pat419_0001_9.png',
 '/content/drive/My Drive/brats_png/labels/mask_pat419_0001_10.png',
 '/content/drive/My Drive/brats_png/labels/mask_pat419_0001_11.png',
 '/content/drive/My Drive/brats_png/labels/mask_pat419_0001_12.png',
 '/content/drive/My Drive/brats_png/labels/mask_pat419_0001_13.png']

The algorithm for extracting binary pattern features is similar to convolution using a window of some size smaller than the image dimensions. The size of a binary pattern window is generally 3x3 and compares the middle values with surrounding values and assigns a 0,1 to the surrounding elements based on the threshold stipulated. This approach is great for edge detection. 

In [0]:
################################################################
def create_binary_pattern(img, p, r):
    print ('Computing local binary pattern features.')
    lbp = feature.local_binary_pattern(img, p, r)

    return (lbp-np.min(lbp))/(np.max(lbp)-np.min(lbp)) * 255

Train and test split occurs in this method. We have our X list and y list. We enumerate in a for loop, converting all our images to gray scale. We use create_features() to engineer all our features. Keep in mind that the images become 1d vectors and we perform operations on these 1d vectors to generate the features. 
We reshape the dimensions of the X set and y set. The y set must be 1d so we reshape it as such with ravel() and the X must be 2d so we do the following operation: 

X.shape[0]*X.shape[1], X.shape[2]

In [0]:
################################################################
def create_training_dataset(image_list, label_list):
  #print ('Creating training dataset on %d image(s).' %len(image_list))

  X = []
  y = []

  for i, img in enumerate(image_list):
    ##print('\n')
    ##print(i,len(img))

    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lab = cv2.cvtColor(label_list[i], cv2.COLOR_BGR2GRAY)
    features, labels = create_features(img, img_gray, lab)
    #features, labels = create_features(img, img_gray, label_list[i])
    ##print('features.shape: ', features.shape)
    ##print('labels.shape: ', labels.shape)
    X.append(features)
    y.append(labels)

  X = np.array(X)
  ##print('X.shape AFTER np.array: ', X.shape)
  X = X.reshape(X.shape[0]*X.shape[1], X.shape[2])
  ##print('X.shape AFTER reshape: ', X.shape)
  y = np.array(y)
  ##print('y.shape AFTER np.array: ', y.shape)
  y = y.reshape(y.shape[0]*y.shape[1], y.shape[2]).ravel()
  ##print('y.shape AFTER np.array: ', y.shape)

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  ##print ('Feature vector size:', X_train.shape)

  return X_train, X_test, y_train, y_test

The range for the binary pattern and haralick neighborhood are defined here. The features are generated after converting the images to grey scale. We accept as parameters the original image, labels and grey scale image. We use the gray scale image to generate binary pattern features  We then randomly sample from the binary pattern feature and stack the features as row vectors. The features and all images will have the same row dimension. We return the features after stacking and subset the labels as we did with the features. 

In [0]:
################################################################
def create_features(img, img_gray, label, train=True):
    lbp_radius = 24 # local binary pattern neighbourhood
    h_neigh = 11 # haralick neighbourhood
    num_examples = 1000 #number of examples per image to use for training model

    lbp_points = lbp_radius*8
    h_ind = int((h_neigh - 1)/ 2)

    feature_img = np.zeros((img.shape[0],img.shape[1],4))
    feature_img[:,:,:3] = img
    ##print('feature_img[:,:,3].shape BEFORE: ', feature_img[:,:,3].shape)


    img = None

    feature_img[:,:,3] = create_binary_pattern(img_gray, lbp_points, lbp_radius)
    ##print('\nfeature_img[:,:,3].shape AFTER: ', feature_img[:,:,3].shape)
    ##print('feature_img BEFORE: ', feature_img.shape)
    feature_img = feature_img[h_ind:-h_ind, h_ind:-h_ind]
    ##print('feature_img AFTER: ', feature_img.shape)
    features = feature_img.reshape(feature_img.shape[0]*feature_img.shape[1], feature_img.shape[2])
    ##print('features.shape BEFORE: ', features.shape)

    if train == True:
        ss_idx = subsample_idx(0, features.shape[0], num_examples)
        features = features[ss_idx]
        ##print('features AFTER features[ss_idx]: ', features.shape)

    else:
        ss_idx = []

    h_features = haralick_features(img_gray, h_neigh, ss_idx)
    ##print('\nh_features.shape: ', h_features.shape)
    features = np.hstack((features, h_features))
    ##print('features.shape AFTER hstack: ', features.shape)

    if train == True:

        label = label[h_ind:-h_ind, h_ind:-h_ind]
        ##print('label BEFORE: ', label.shape)
        ##print('label.shape[0]*label.shape[1]: ', label.shape[0]*label.shape[1])
        labels = label.reshape(label.shape[0]*label.shape[1], 1)
        #labels = label.reshape(label.shape[0]*label.shape[1], label.shape[2])
        labels = labels[ss_idx]
        
    else:
        labels = None

    return features, labels

In [0]:
# i_list, l_list should be matrices from cv2.imread(), not file paths
X_train, X_test, y_train, y_test = create_training_dataset(image_list, label_list)

#binary pattern features: (52900, 4)
#haralick features: (1000, 13)

#230x230x3 = 158700
#230x230 = 52900
#ValueError: cannot reshape array of size 158700 into shape (52900,1)

Computing local binary pattern features.


[==                                                                      ]   3%

Computing haralick features.


[========================================================================] 100%

Computing local binary pattern features.


...

[========================================================================] 100%

Computing local binary pattern features.


[==                                                                      ]   3%

Computing haralick features.


[========================================================================] 100%

These are our results. Of course, more tests need to be done to secure this kind of accuracy. This is just a single test on a small sample of 2000 images out of 50000 images. 

Summary: If the use of engineered features such as haralick and binary pattern features are capable of churning out good results as these on image data sets, it is good idea to continue pursuing such techniques before resorting to neural nets. The only concern with traditional machine learning algorithms is that they perform quite slow on image sets. The only hope of training image sets in good time is to use several GPUs in parallel. 

In [0]:
# 2000 images
start = time.time()
model = train_model(X_trn,y_trn,'RF')
test_model(X_tst,y_tst,model)
print ('Processing time:',time.time()-start)

Training Random Forest model.
Model training complete.
Training Accuracy: 0.99
--------------------------------
Accuracy: 0.99
Precision: 0.98
Recall: 0.99
F1: 0.98
--------------------------------
Processing time: 468.48119926452637
